In [1]:
#### Loading the packages to be used
import pandas as pd
import numpy as np
from __future__ import print_function  # Python 2 and 3
#from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms

In [2]:
drugDF= pd.read_csv('drug.csv',dtype={'NDC': str})
drugDF.columns
drugDF.head()

,PRODUCTID_x,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,PRODUCTID_y,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,...,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,0009-3073_0f1820e6-28cb-4dd7-89ee-00583ebab35d,Sep-73,0009-3073-01,"1 VIAL, SINGLE-DOSE in 1 PACKAGE (0009-3073-01...",0009-3073_0f1820e6-28cb-4dd7-89ee-00583ebab35d,HUMAN PRESCRIPTION DRUG,Depo-Medrol,NaN,methylprednisolone acetate,"INJECTION, SUSPENSION",...,0.02878,05/17/2017,EA,C/I,N,1,G,NaN,NaN,06/07/2017
1,0009-3073_0f1820e6-28cb-4dd7-89ee-00583ebab35d,Sep-73,0009-3073-01,"1 VIAL, SINGLE-DOSE in 1 PACKAGE (0009-3073-01...",0009-3073_48526b2b-3cef-4d67-8c38-d3e9fca1cf3d,HUMAN PRESCRIPTION DRUG,Depo-Medrol,NaN,methylprednisolone acetate,"INJECTION, SUSPENSION",...,0.02878,05/17/2017,EA,C/I,N,1,G,NaN,NaN,06/07/2017
2,0009-3116_deaa2a92-7be0-4a9a-ae39-6297d868d630,Sep-16,0009-3116-01,1 BOTTLE in 1 CARTON (0009-3116-01) > 30 mL i...,0009-3116_deaa2a92-7be0-4a9a-ae39-6297d868d630,HUMAN PRESCRIPTION DRUG,Cleocin T,NaN,clindamycin phosphate,SOLUTION,...,0.17410,05/17/2017,EA,C/I,N,1,G,NaN,NaN,06/07/2017
3,10019-553_9caea3f4-e7e2-4d24-b3b6-5e62f37b3ec8,10019-553,10019-553-01,10 POUCH in 1 BOX (10019-553-01) > 1 PATCH in...,10019-553_9caea3f4-e7e2-4d24-b3b6-5e62f37b3ec8,HUMAN PRESCRIPTION DRUG,Transderm Scop,NaN,Scopalamine,"PATCH, EXTENDED RELEASE",...,20.40381,03/22/2017,EA,C/I,N,"2, 5",B,NaN,NaN,06/07/2017
4,10019-553_9caea3f4-e7e2-4d24-b3b6-5e62f37b3ec8,10019-553,10019-553-01,10 POUCH in 1 BOX (10019-553-01) > 1 PATCH in...,10019-553_b877a694-a1d0-4280-937a-a06820b12a88,HUMAN PRESCRIPTION DRUG,Transderm Scop,NaN,Scopalamine,"PATCH, EXTENDED RELEASE",...,20.40381,03/22/2017,EA,C/I,N,"2, 5",B,NaN,NaN,06/07/2017


In [3]:
pd.to_numeric(drugDF.STARTMARKETINGDATE)

0        19590528.0
1        19590528.0
2        19800620.0
3        20030101.0
4        20161201.0
5        20030101.0
6        20161201.0
7        20030101.0
8        20161201.0
9        20030101.0
10       20161201.0
11       20030101.0
12       20161201.0
13       20030101.0
14       20161201.0
15       20070530.0
16       19881230.0
17       19881230.0
18       19920930.0
19       20021220.0
20       20071214.0
21       20140707.0
22       20140707.0
23       20070612.0
24       20070612.0
25       20070612.0
26       20070612.0
27       20070612.0
28       20111206.0
29       20110414.0
            ...    
13072    20140325.0
13073    20140325.0
13074    20140325.0
13075    20140325.0
13076    20140325.0
13077    20140325.0
13078    20140325.0
13079    20131211.0
13080    20140409.0
13081    20131211.0
13082    20140409.0
13083    20141121.0
13084    20140404.0
13085    20140404.0
13086    20140401.0
13087    20120608.0
13088    20110904.0
13089    20110904.0
13090    20110904.0


In [4]:
#drugPrice=drugDF.copy()

keyfeatures=(['DOSAGEFORMNAME','ROUTENAME','LABELERNAME','STARTMARKETINGDATE','MARKETINGCATEGORYNAME',
            'PHARM_CLASSES','DEASCHEDULE','OTC','Classification_for_Rate_Setting','NADAC_Per_Unit'])

drugPrice = drugDF.loc[:,keyfeatures]

In [5]:
def Dummify(column):
    column_values = pd.get_dummies(drugPrice[column], prefix=column, prefix_sep='__')
    return pd.concat([drugPrice.drop(column,axis=1),column_values],axis=1)

drugPrice = Dummify('DOSAGEFORMNAME')
drugPrice = Dummify('ROUTENAME')
drugPrice = Dummify('LABELERNAME')
drugPrice = Dummify('PHARM_CLASSES')
drugPrice = Dummify('DEASCHEDULE')
drugPrice = Dummify('OTC')
drugPrice = Dummify('Classification_for_Rate_Setting')
drugPrice = Dummify('MARKETINGCATEGORYNAME')

In [6]:
drugPrice.shape

(13102, 1145)

In [7]:
Y = drugPrice.pop('NADAC_Per_Unit')
X = drugPrice

In [8]:
X_train,X_test,y_train,y_test = ms.train_test_split(X,Y,test_size=0.2)

#X.iloc[X_train] # return dataframe train
X_train
X_test

,STARTMARKETINGDATE,"DOSAGEFORMNAME__AEROSOL, FOAM","DOSAGEFORMNAME__AEROSOL, METERED","DOSAGEFORMNAME__AEROSOL, POWDER","DOSAGEFORMNAME__AEROSOL, SPRAY",DOSAGEFORMNAME__CAPSULE,"DOSAGEFORMNAME__CAPSULE, COATED PELLETS","DOSAGEFORMNAME__CAPSULE, COATED, EXTENDED RELEASE","DOSAGEFORMNAME__CAPSULE, DELAYED RELEASE","DOSAGEFORMNAME__CAPSULE, DELAYED RELEASE PELLETS",...,Classification_for_Rate_Setting__B-ANDA,Classification_for_Rate_Setting__G,MARKETINGCATEGORYNAME__ANDA,MARKETINGCATEGORYNAME__BLA,MARKETINGCATEGORYNAME__NDA,MARKETINGCATEGORYNAME__NDA AUTHORIZED GENERIC,MARKETINGCATEGORYNAME__OTC MONOGRAPH FINAL,MARKETINGCATEGORYNAME__OTC MONOGRAPH NOT FINAL,MARKETINGCATEGORYNAME__UNAPPROVED DRUG OTHER,MARKETINGCATEGORYNAME__UNAPPROVED OTHER
5801,20160803.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
10308,20110120.0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
270,20091025.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3155,20141101.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1283,20140516.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
10371,20151221.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
12525,20160722.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
8832,20121101.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
12889,20150414.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
11695,20081113.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [10]:
regressor = RandomForestRegressor(n_estimators= 100 , oob_score =True, random_state = 42)
regressor.fit(X_train,y_train)
y_pred=regressor.predict(X_test)

In [12]:
#from sklearn.metrics import mean_squared_error
#rms = np.sqrt(mean_squared_error(y_actual, y_predicted))

errorScore=np.sqrt(((y_pred - y_test) ** 2).mean())
print("Root Mean Squared error: %.2f"
      % errorScore)

Root Mean Squared error: 114.69
